In [1]:
from __future__ import print_function, division

# import sys
# sys.path.append(root_folder)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pathlib

from models.convengers import ResNet, Inception, SeaNormaBlock
from utils import dictionary, visualizations

import GPUtil

plt.ion()   # interactive mode

%load_ext autoreload
%autoreload 2

In [2]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 95% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 |  0% |  0% |


In [3]:
dataset_folder = "tiny-imagenet-200/"
data_dir = pathlib.Path(dataset_folder)

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
image_datasets = {x: datasets.ImageFolder(data_dir / x, data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=20, shuffle=True, num_workers=4, pin_memory=False) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [14]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        GPUtil.showUtilization()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_total = 0

            # Iterate over data.
            for idx, (inputs, labels) in enumerate(dataloaders[phase]):
                
                with torch.cuda.device(torch.cuda.current_device()):
                    torch.cuda.empty_cache()
                    
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_total += labels.size(0)
                
                print("\r", end='')
                print(f'training {100 * idx / len(dataloaders[phase]):.2f}%: {running_corrects / running_total:.3f}', end='')
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
with torch.cuda.device(torch.cuda.current_device()):
    torch.cuda.empty_cache()
    
resnet_new = ResNet()
resnet_new = resnet_new.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(resnet_new.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [7]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 95% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 |  0% |  5% |


In [8]:
resnet_new = train_model(resnet_new, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=10)

Epoch 0/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 95% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 |  1% |  5% |
training 5.00%: 0.281train Loss: 3.3278 Acc: 0.2814
training 4.99%: 0.533val Loss: 1.9320 Acc: 0.5334

Epoch 1/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 | 43% |  7% |
training 5.00%: 0.349train Loss: 2.9057 Acc: 0.3489
training 4.99%: 0.562val Loss: 1.8000 Acc: 0.5620

Epoch 2/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
|  1 |  0% | 91% |
|  2 |  8% | 64% |
|  3 | 45% |  8% |
training 5.00%: 0.363train Loss: 2.8322 Acc: 0.3629
training 4.99%: 0.562val Loss: 1.7817 Acc: 0.5617

Epoch 3/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 | 89% |  8% |
training 5.00%: 0.366train Loss: 2.8067 Acc: 0.3660
training 4.99%: 0.569val Loss: 1.7497 Acc: 0.5693

Epoch 4/9
----------
| ID | GPU | MEM |
--------

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



training 5.00%: 0.398train Loss: 2.6325 Acc: 0.3983
training 4.99%: 0.598val Loss: 1.6432 Acc: 0.5984

Epoch 9/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 | 70% |  8% |
training 5.00%: 0.400train Loss: 2.6207 Acc: 0.3997
training 4.99%: 0.601val Loss: 1.6069 Acc: 0.6015

Training complete in 48m 41s
Best val Acc: 0.603600


In [12]:
with torch.cuda.device(torch.cuda.current_device()):
    torch.cuda.empty_cache()
    
inception_new = Inception()
inception_new = inception_new.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(inception_new.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [15]:
inception_new = train_model(inception_new, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=10)

Epoch 0/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 |  1% | 12% |
training 99.98%: 0.156train Loss: 4.2188 Acc: 0.1564
training 99.80%: 0.436val Loss: 2.5549 Acc: 0.4357

Epoch 1/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 | 82% | 13% |
training 99.98%: 0.238train Loss: 3.6436 Acc: 0.2378
training 99.80%: 0.480val Loss: 2.2499 Acc: 0.4797

Epoch 2/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 | 29% | 13% |
training 99.98%: 0.251train Loss: 3.5745 Acc: 0.2509
training 99.80%: 0.475val Loss: 2.2791 Acc: 0.4746

Epoch 3/9
----------
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
|  1 |  0% | 91% |
|  2 |  0% | 64% |
|  3 | 93% | 13% |
training 99.98%: 0.258train Loss: 3.5405 Acc: 0.2583
training 99.80%: 0.490val Loss: 2.1992 Acc: 0.4900

Epoch 4/9
----------
| ID | GPU | MEM |


In [22]:
images, labels = next(iter(dataloaders["val"]))
resnet_new.to("cpu")
inception_new.to("cpu")
with torch.no_grad():
    resnet_preds = resnet_new(images)
    inception_preds = inception_new(images)

print(labels)
print(torch.argmax(resnet_preds,dim=1))
print(torch.argmax(inception_preds,dim=1))

tensor([145, 120,  44,  32,  68, 139, 148,  99,  29,  74, 102, 133, 150,  79,
        156,  53, 131,  75,  69,  44])
tensor([145, 120,  44,  16, 171, 139,  41,  99,  35, 158, 102, 121, 152,  79,
        170,  53, 131,  84,  69,  44])
tensor([ 64, 120,  44,  16, 171,  99,  11, 109,  35, 158, 102, 133,  16, 173,
        143,  53, 177,  75,  69,  44])
